In [1]:
import pandas as pd
from collections import defaultdict
import re

In [2]:
df = pd.read_csv('/work/InternalMedicine/s223850/ED-StaticDynamic/raw_data/ED Events - 12.1.23.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16832198 entries, 0 to 16832197
Data columns (total 30 columns):
 #   Column                                                                  Dtype  
---  ------                                                                  -----  
 0   PAT_ENC_CSN_ID                                                          int64  
 1   PAT_MRN_ID                                                              int64  
 2   PAT_ID                                                                  object 
 3   Ethnicity                                                               object 
 4   FirstRace                                                               object 
 5   MultiRacial                                                             int64  
 6   Sex                                                                     object 
 7   Arrived_Time                                                            object 
 8   ED_Disposition                

In [4]:
df['Calculated_DateTime'] = pd.to_datetime(df['Calculated_DateTime'])
df['Arrived_Time'] = pd.to_datetime(df['Arrived_Time'])

In [10]:
df = df.sort_values(by='Calculated_DateTime')

In [11]:
daily_diff = (df['Calculated_DateTime']-df['Arrived_Time']).dt.total_seconds()/(60*60*24)
(daily_diff > 3).sum()

8580

In [12]:
daily_diff[daily_diff>3].index

Int64Index([11190203,  8435724,  8435725, 16513490, 10228257, 16513491,
            13275353,  8435726, 10228258,  8435727,
            ...
            15371184,  4989354, 10667005, 10667002, 10667004,  4295462,
            15324049, 12013116,  4989353, 15371301],
           dtype='int64', length=8580)

In [16]:
df_first_group = df.groupby('PAT_ENC_CSN_ID')[['Calculated_DateTime','Arrived_Time','EVENT_NAME','Type']].first()

In [21]:
df_first_group

,Calculated_DateTime,Arrived_Time,EVENT_NAME,Type,time_diff
PAT_ENC_CSN_ID,,,,,
86725001,2017-04-02 09:58:23,2022-03-03 11:53:00,Prev Admission Rec Generated,Event,-1796.079595
619927095,2019-11-14 10:18:28,2022-03-03 11:43:00,Emergency encounter created,Event,-840.058704
633157523,2020-11-05 14:18:36,2021-11-08 19:50:00,Emergency encounter created,Event,-368.230139
651542751,2021-11-01 00:44:25,2021-11-01 00:44:00,Emergency encounter created,Event,0.000289
651542792,2021-11-01 00:53:45,2021-11-01 00:53:45,Emergency encounter created,Event,0.000000
...,...,...,...,...,...
693923279,2023-11-30 19:49:44,2023-11-30 19:49:44,Emergency encounter created,Event,0.000000
693923415,2023-11-30 20:01:12,2023-11-30 20:01:00,Emergency encounter created,Event,0.000139
693924042,2023-11-30 21:08:27,2023-11-30 21:08:00,Emergency encounter created,Event,0.000313


In [19]:
df_first_group['time_diff'] = (df_first_group['Calculated_DateTime']-df_first_group['Arrived_Time']).dt.total_seconds()/(60*60*24)

In [27]:
df_first_group[(df_first_group['time_diff'] > 15) | (df_first_group['time_diff'] < -15)]

,Calculated_DateTime,Arrived_Time,EVENT_NAME,Type,time_diff
PAT_ENC_CSN_ID,,,,,
86725001,2017-04-02 09:58:23,2022-03-03 11:53:00,Prev Admission Rec Generated,Event,-1796.079595
619927095,2019-11-14 10:18:28,2022-03-03 11:43:00,Emergency encounter created,Event,-840.058704
633157523,2020-11-05 14:18:36,2021-11-08 19:50:00,Emergency encounter created,Event,-368.230139
662549179,2022-06-28 09:40:00,2022-05-24 10:46:06,ACETAMINOPHEN-CODEINE 300 MG-30 MG TAB,Order - Medication,34.954097


In [5]:
df_uniq_cnts = df.groupby('PAT_ENC_CSN_ID').nunique()

In [6]:
(df_uniq_cnts<=1).sum()

PAT_MRN_ID                                                                122420
PAT_ID                                                                    122420
Ethnicity                                                                 122420
FirstRace                                                                 122420
MultiRacial                                                               122420
Sex                                                                       122420
Arrived_Time                                                              122420
ED_Disposition                                                            122420
Admitted_YN                                                               122420
Patient_Age                                                               122420
Acuity_Level                                                              122420
Means_Of_Arrival                                                          122420
Chief_Complaint             

In [63]:
df['Type'].value_counts()


Event                 7009562
Lab Order - Result    5288796
Order - Lab           1853181
Order - Nursing        955960
Order - Medication     775114
Order - Imaging        551962
Order - Consult         84933
Order - Admission       81446
Order - Procedures      11585
Order - Discharge         320
Name: Type, dtype: int64

In [18]:
df['PAT_ENC_CSN_ID'].nunique()

120877

In [19]:
target = ['Admitted_YN']

In [28]:
stationary_fields = ['PAT_ENC_CSN_ID','PAT_MRN_ID', 'PAT_ID', 'Ethnicity', 'Sex', 'MultiRacial', 'Arrived_Time', 'Admitted_YN', 'Patient_Age', 
                     'Coverage_Financial_Class_Grouper', 'Has Completed Appt in Last Seven Days', 'Has Hospital Encounter in Last Seven Days',
                     'Number of Inpatient Admissions in the last 30 Days', 'Number of past appointments in last 60 days', 'FirstRace',
                     'Number of past inpatient admissions over ED visits in last three years', 'Chief_Complaint_All', 'Count_of_Chief_Complaints', 'Means_Of_Arrival',
                    'Acuity_Level']

In [8]:
dynamic_fields = ['PAT_ENC_CSN_ID', 'Type', 'EVENT_NAME', 'Order_Status', 'Result_Flag', 'Primary_DX_Name', 'Primary_DX_ICD10', 'Calculated_DateTime']

In [9]:
dropped_fields = ['PAT_MRN_ID', 'PAT_ID']

In [10]:
df['Admitted_YN'].isna().sum()

0

# Stationary fields analysis

In [29]:
df_stationary = df[stationary_fields].drop(columns=dropped_fields)

In [30]:
df_stationary.head()

,PAT_ENC_CSN_ID,Ethnicity,Sex,MultiRacial,Arrived_Time,Admitted_YN,Patient_Age,Coverage_Financial_Class_Grouper,Has Completed Appt in Last Seven Days,Has Hospital Encounter in Last Seven Days,Number of Inpatient Admissions in the last 30 Days,Number of past appointments in last 60 days,FirstRace,Number of past inpatient admissions over ED visits in last three years,Chief_Complaint_All,Count_of_Chief_Complaints,Means_Of_Arrival,Acuity_Level
0,663857849,Non-Hispanic/Latino,Male,0,2022-06-18 21:18:51,Not Admitted,74.81451,Self-Pay,Yes,No,0,17,White,0,SYNCOPE,1,Car,II
1,663857849,Non-Hispanic/Latino,Male,0,2022-06-18 21:18:51,Not Admitted,74.81451,Self-Pay,Yes,No,0,17,White,0,SYNCOPE,1,Car,II
2,663857849,Non-Hispanic/Latino,Male,0,2022-06-18 21:18:51,Not Admitted,74.81451,Self-Pay,Yes,No,0,17,White,0,SYNCOPE,1,Car,II
3,663857849,Non-Hispanic/Latino,Male,0,2022-06-18 21:18:51,Not Admitted,74.81451,Self-Pay,Yes,No,0,17,White,0,SYNCOPE,1,Car,II
4,663857849,Non-Hispanic/Latino,Male,0,2022-06-18 21:18:51,Not Admitted,74.81451,Self-Pay,Yes,No,0,17,White,0,SYNCOPE,1,Car,II


In [31]:
df_st_grb = df_stationary.groupby('PAT_ENC_CSN_ID')

In [32]:
data_dict = dict()
for pat_id, df_grb in df_st_grb:
    for col in df_grb.columns:
        assert(df_grb[col].nunique() <= 1)
    data_dict[pat_id] = df_grb.iloc[-1].tolist()
    

In [33]:
df_stat_mat = pd.DataFrame(data_dict, index=[df_grb.columns]).T

In [34]:
df_stat_mat.drop('PAT_ENC_CSN_ID', axis=1, inplace=True)

/tmp/ipykernel_35357/1290416507.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_stat_mat.drop('PAT_ENC_CSN_ID', axis=1, inplace=True)


In [17]:
for col in df_stat_mat.columns:
    print(f"Number of unique values in {col} is {df_stat_mat[col].nunique()} ...")

Number of unique values in ('Ethnicity',) is 9 ...
Number of unique values in ('Sex',) is 4 ...
Number of unique values in ('MultiRacial',) is 2 ...
Number of unique values in ('Arrived_Time',) is 120628 ...
Number of unique values in ('Admitted_YN',) is 2 ...
Number of unique values in ('Patient_Age',) is 28292 ...
Number of unique values in ('Coverage_Financial_Class_Grouper',) is 7 ...
Number of unique values in ('Has Completed Appt in Last Seven Days',) is 2 ...
Number of unique values in ('Has Hospital Encounter in Last Seven Days',) is 2 ...
Number of unique values in ('Number of Inpatient Admissions in the last 30 Days',) is 97 ...
Number of unique values in ('Number of past appointments in last 60 days',) is 491 ...
Number of unique values in ('Number of past inpatient admissions over ED visits in last three years',) is 44 ...
Number of unique values in ('Chief_Complaint_All',) is 10216 ...
Number of unique values in ('Count_of_Chief_Complaints',) is 9 ...
Number of unique valu

In [35]:
df_stat_mat['FirstRace'].value_counts()

(FirstRace,)                             
White                                        65697
Black or African American                    40610
Unavailable/Unknown                           7367
Asian                                         3757
Some other race                               2914
American Indian or Alaska Native               529
Declined                                       432
Native Hawaiian or Other Pacific Islander      144
dtype: int64

In [18]:
df['Acuity_Level'].value_counts()

II     8272447
III    6344532
IV      563246
I       312590
V        19940
Name: Acuity_Level, dtype: int64

In [19]:
df_stat_mat['Acuity_Level'].value_counts()

(Acuity_Level,)
II                 47604
III                46741
IV                  9088
I                   1485
V                    611
dtype: int64

In [20]:
df['Sex'].value_counts()

Female          9522299
Male            7308906
Unknown             950
*Unspecified         43
Name: Sex, dtype: int64

In [21]:
df['Means_Of_Arrival'].value_counts()

Car                      13316105
Ambulance                 2889539
Other                      189342
Wheelchair                 148634
Pedestrian                  80131
Assist from Vehicle         40412
Police                      39796
Public Transportation       35361
Helicopter                  27688
Hospital Transport          20586
Fixed Wing                  16011
Taxi                        15796
Name: Means_Of_Arrival, dtype: int64

In [22]:
df['Ethnicity'].value_counts()

Non-Hispanic/Latino                 13257472
Hispanic or Latino                   3178852
Unknown                               269799
Declined                              105111
*Unspecified                           13025
American Indian or Alaska Native        3182
White                                   2831
Black or African American               1802
Asian                                    124
Name: Ethnicity, dtype: int64

In [23]:
df['Coverage_Financial_Class_Grouper'].value_counts()

Commercial    5313280
Medicaid      4319466
Self-Pay      3426114
Medicare      1631902
None          1437323
Other          432437
Exchange       271676
Name: Coverage_Financial_Class_Grouper, dtype: int64

In [24]:
df['Coverage_Financial_Class_Grouper'].isna().sum()

0

In [25]:
df.isna().sum()

PAT_ENC_CSN_ID                                                                   0
PAT_MRN_ID                                                                       0
PAT_ID                                                                           0
Ethnicity                                                                        0
FirstRace                                                                    11689
MultiRacial                                                                      0
Sex                                                                              0
Arrived_Time                                                                     0
ED_Disposition                                                               40993
Admitted_YN                                                                      0
Patient_Age                                                                     77
Acuity_Level                                                               1319443
Mean

# Get the correlated flags

In [41]:
df['EVENT_NAME_NORM'] = df['EVENT_NAME'].apply(lambda x: re.sub(r'[^a-zA-Z0-9_]', '', x))

In [43]:
admi_df = df.loc[df['EVENT_NAME_NORM'].str.lower().str.contains('admi'), ['EVENT_NAME', 'EVENT_NAME_NORM', 'Admitted_YN']]

In [44]:
admi_val_cnt = admi_df.groupby('Admitted_YN').value_counts()

In [62]:
data_norm = pd.DataFrame(0, index=df['EVENT_NAME_NORM'].unique(), columns=['Admitted', 'Not Admitted'])
data = pd.DataFrame(0, index=df['EVENT_NAME'].unique(), columns=['Admitted', 'Not Admitted'])

for (target, event, event_norm), value in admi_val_cnt.items():
    data.loc[event, target] += value
    data_norm.loc[event_norm, target] = value


In [53]:
data_norm['sum'] = data_norm.sum(axis=1)

In [54]:
data_norm = data_norm[data_norm['sum']!=0]

In [55]:
data_norm['perc_admit'] = data_norm['Admitted']/data_norm['sum']
data_norm['perc_nonadmit'] = data_norm['Not Admitted']/data_norm['sum']


/tmp/ipykernel_4668/3165786617.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_norm['perc_admit'] = data_norm['Admitted']/data_norm['sum']
/tmp/ipykernel_4668/3165786617.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_norm['perc_nonadmit'] = data_norm['Not Admitted']/data_norm['sum']


In [56]:
data_norm = data_norm.sort_values(by='perc_admit', ascending=False)

In [58]:
data_norm.loc[(data_norm['perc_admit']>=0.70)|(data_norm['perc_admit']<=0.30)].index

Index(['ConsultCardiologyAdmitsForUseByEDProvidersOnly',
       'EDCUHHospitalistCOVIDAdmitsCompleted',
       'NMRADIOPHARMACEUTICALIVTHERAPYADMINISTRATION',
       'ConsultOrderedCONSULTCUHPULMMICUCOVIDADMITS',
       'HospitalistAdmitsConsultCompleted', 'HospitalistAdmitsConsultInED',
       'CHFVADHeartTXAdmitsReCalled',
       'ConsultCHFVADHeartTXAdmitsForUseByEDProvidersOnly',
       'CHFVADHeartTXAdmitsRespond', 'CHFVADHeartTXAdmitsCompleted',
       'CUHPulmMICUCOVIDAdmitsReCalled', 'AdmissionOrderRecPrivatePend',
       'ADMITASOBSERVATION', 'ZaleAdmitOrderPlaced',
       'CUHPulmMICUCOVIDAdmitsConsultCalled', 'UHAdmitOrderPlaced',
       'HospitalistAdmitsConsultReCalled',
       'PHARMACYTECHCONSULTADMISSIONMEDICATIONHISTORY', 'ADMITASINPATIENT',
       'POTENTIALADMISSIONBEDBOARDNOTIFICATION', 'AdmitStatusChange',
       'AdmissionOrdersReconciled', 'HospitalistAdmitsConsultCalled',
       'HospitalistAdmitsConsultOrdered',
       'ConsultOrderedCONSULTCUHHOSPITALISTSCOVID

# Calculated_DateTime

In [64]:
 l = [86725001, 619927095, 657949013, 658712595, 668214622, 669861703,
       682818856]

In [70]:
df[df['PAT_ENC_CSN_ID'] == l[4]][['Arrived_Time', 'Calculated_DateTime', 'PAT_MRN_ID']]

,Arrived_Time,Calculated_DateTime,PAT_MRN_ID
6881445,2022-09-11 15:18:55.000,2022-09-11 15:18:55.000,92441846
6881446,2022-09-11 15:18:55.000,2022-09-11 15:19:13.000,92441846
6881447,2022-09-11 15:18:55.000,2022-09-11 15:19:13.000,92441846
6881448,2022-09-11 15:18:55.000,2022-09-11 15:19:13.000,92441846
6881449,2022-09-11 15:18:55.000,2022-09-11 15:19:13.000,92441846
...,...,...,...
6881637,2022-09-11 15:18:55.000,2022-09-11 23:48:00.000,92441846
6881638,2022-09-11 15:18:55.000,2022-09-11 23:48:00.000,92441846
6881639,2022-09-11 15:18:55.000,2022-09-11 23:48:00.000,92441846
6881640,2022-09-11 15:18:55.000,2022-09-11 23:48:00.000,92441846


In [ ]:
df